In [1]:
import json
import os
import faiss
import numpy as np
from fuzzywuzzy import fuzz
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

# Set your OpenAI API key
openai_api_key = 'sk-proj-'

In [2]:
# Toolset data as JSON

# Load toolset from tools.json file
# with open("tools.json", "r") as file:
#     toolset = json.load(file)

toolset_json = '''
[
    {
        "tool_name": "who_am_i",
        "tool_description": "Returns the id of the current user",
        "args": [],
        "output": {
            "arg_type": "str",
            "is_array": false,
            "is_required": true
        }
    },
    {
        "tool_name": "get_sprint_id",
        "tool_description": "Returns the ID of the current sprint",
        "args": [],
        "output": {
            "arg_type": "str",
            "is_array": false,
            "is_required": true
        }
    },
    {
        "tool_name": "works_list",
        "tool_description": "Returns a list of work items matching the request",
        "args": [
            {"arg_name": "applies_to_part", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "created_by", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "issue_priority", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "issue.rev_orgs", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "limit", "arg_type": "int", "is_array": false, "is_required": false},
            {"arg_name": "owned_by", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "stage_name", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "ticket_need_response", "arg_type": "boolean", "is_array": false, "is_required": false},
            {"arg_name": "ticket_rev_org", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "ticket_severity", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "ticket_source_channel", "arg_type": "str", "is_array": true, "is_required": false},
            {"arg_name": "type", "arg_type": "str", "is_array": true, "is_required": false}
        ],
        "output": {
            "arg_type": "any",
            "is_array": false,
            "is_required": true
        }
    },
    {
        "tool_name": "summarize_objects",
        "tool_description": "Summarizes a list of objects.",
        "args": [
            {"arg_name": "objects", "arg_type": "any", "is_array": true, "is_required": true}
        ],
        "output": {
            "arg_type": "any",
            "is_array": false,
            "is_required": true
        }
    },
    {
        "tool_name": "prioritize_objects",
        "tool_description": "Returns a list of objects sorted by priority.",
        "args": [
            {"arg_name": "objects", "arg_type": "any", "is_array": false, "is_required": false}
        ],
        "output": {
            "arg_type": "any",
            "is_array": true,
            "is_required": true
        }
    },
    {
        "tool_name": "add_work_items_to_sprint",
        "tool_description": "Adds the given work items to the sprint",
        "args": [
            {"arg_name": "work_ids", "arg_type": "str", "is_array": true, "is_required": true},
            {"arg_name": "sprint_id", "arg_type": "str", "is_array": false, "is_required": true}
        ],
        "output": {
            "arg_type": "boolean",
            "is_array": false,
            "is_required": true
        }
    },
    {
        "tool_name": "get_similar_work_items",
        "tool_description": "Returns a list of work items that are similar to the given work item",
        "args": [
            {"arg_name": "work_id", "arg_type": "str", "is_array": false, "is_required": true}
        ],
        "output": {
            "arg_type": "str",
            "is_array": true,
            "is_required": true
        }
    },
    {
        "tool_name": "search_object_by_name",
        "tool_description": "Given a search string, returns the id of a matching object in the system of record.",
        "args": [
            {"arg_name": "query", "arg_type": "str", "is_array": false, "is_required": true}
        ],
        "output": {
            "arg_type": "any",
            "is_array": false,
            "is_required": true
        }
    },
    {
        "tool_name": "create_actionable_tasks_from_text",
        "tool_description": "Given a text, extracts actionable insights and creates tasks for them.",
        "args": [
            {"arg_name": "text", "arg_type": "str", "is_array": false, "is_required": true}
        ],
        "output": {
            "arg_type": "str",
            "is_array": true,
            "is_required": true
        }
    }
]
'''
toolset  = [
    {
        "tool_name": "who_am_i",
        "tool_description": "Returns the id of the current user",
        "args": [],
        "output": {
            "arg_type": "str",
            "is_array": False,
            "is_required": True
        }
    },
    {
        "tool_name": "get_sprint_id",
        "tool_description": "Returns the ID of the current sprint",
        "args": [],
        "output": {
            "arg_type": "str",
            "is_array": False,
            "is_required": True
        }
    },
    {
        "tool_name": "works_list",
        "tool_description": "Returns a list of work items matching the request",
        "args": [
            {"arg_name": "applies_to_part", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "created_by", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "issue_priority", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "issue.rev_orgs", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "limit", "arg_type": "int", "is_array": False, "is_required": False},
            {"arg_name": "owned_by", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "stage_name", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "ticket_need_response", "arg_type": "boolean", "is_array": False, "is_required": False},
            {"arg_name": "ticket_rev_org", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "ticket_severity", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "ticket_source_channel", "arg_type": "str", "is_array": True, "is_required": False},
            {"arg_name": "type", "arg_type": "str", "is_array": True, "is_required": False}
        ],
        "output": {
            "arg_type": "any",
            "is_array": False,
            "is_required": True
        }
    },
    {
        "tool_name": "summarize_objects",
        "tool_description": "Summarizes a list of objects.",
        "args": [
            {"arg_name": "objects", "arg_type": "any", "is_array": True, "is_required": True}
        ],
        "output": {
            "arg_type": "any",
            "is_array": False,
            "is_required": True
        }
    },
    {
        "tool_name": "prioritize_objects",
        "tool_description": "Returns a list of objects sorted by priority.",
        "args": [
            {"arg_name": "objects", "arg_type": "any", "is_array": False, "is_required": False}
        ],
        "output": {
            "arg_type": "any",
            "is_array": True,
            "is_required": True
        }
    },
    {
        "tool_name": "add_work_items_to_sprint",
        "tool_description": "Adds the given work items to the sprint",
        "args": [
            {"arg_name": "work_ids", "arg_type": "str", "is_array": True, "is_required": True},
            {"arg_name": "sprint_id", "arg_type": "str", "is_array": False, "is_required": True}
        ],
        "output": {
            "arg_type": "boolean",
            "is_array": False,
            "is_required": True
        }
    },
    {
        "tool_name": "get_similar_work_items",
        "tool_description": "Returns a list of work items that are similar to the given work item",
        "args": [
            {"arg_name": "work_id", "arg_type": "str", "is_array": False, "is_required": True}
        ],
        "output": {
            "arg_type": "str",
            "is_array": True,
            "is_required": True
        }
    },
    {
        "tool_name": "search_object_by_name",
        "tool_description": "Given a search string, returns the id of a matching object in the system of record.",
        "args": [
            {"arg_name": "query", "arg_type": "str", "is_array": False, "is_required": True}
        ],
        "output": {
            "arg_type": "any",
            "is_array": False,
            "is_required": True
        }
    },
    {
        "tool_name": "create_actionable_tasks_from_text",
        "tool_description": "Given a text, extracts actionable insights and creates tasks for them.",
        "args": [
            {"arg_name": "text", "arg_type": "str", "is_array": False, "is_required": True}
        ],
        "output": {
            "arg_type": "str",
            "is_array": True,
            "is_required": True
        }
    }
]

# Load toolset from JSON
toolset = json.loads(toolset_json)

In [3]:

#Calling GPT-3.5 using API KEY

# Function to select the best matching tool based on the query
def find_best_tool(query, toolset):
    # Create a prompt that takes the user query and tools as context
    prompt_template = '''
    You are given a toolset with multiple tools, each having a description and arguments.
    Based on the user's query, identify the most appropriate tool(s) that can fulfill the query.
    Provide the tool's name and the arguments in JSON format, strictly matching the argument names 
    provided in the toolset. Do not create new arguments.

    Available tools and descriptions:
    {toolset}

    User query: "{query}"
    '''

    # Convert toolset to a string format
    formatted_toolset = "\n".join([f"{tool['tool_name']}: {tool['tool_description']}" for tool in toolset])

    # Format the prompt
    prompt = prompt_template.format(toolset=formatted_toolset, query=query)
    
    # Define a ChatGPT-3.5 turbo model instance with the API key
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
    
    # Create an instance of LLMChain
    prompt_template = ChatPromptTemplate.from_template(prompt)
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Prepare inputs for the chain
    inputs = {
        "query": query,  # Pass the user query as input
        "toolset": formatted_toolset  # Pass the formatted toolset
    }

    # Use invoke() to get the tool recommendation
    tool_recommendation = chain.invoke(inputs)
    
    # Print the raw output from the model for debugging
    print("Raw output from the model:", tool_recommendation)
    
    # Extract the text key from the output dictionary
    text_output = tool_recommendation.get('text', '')
    
    # Return the extracted text for further processing
    return text_output

# Function to convert the model output into the desired JSON format
def parse_tool_recommendation(raw_output):
    try:
        # Parse the JSON string from the output
        tool_recommendation_json = json.loads(raw_output)

        # Create the required output format based on the specification
        final_output = []
        for tool_name, arguments in tool_recommendation_json.items():
            formatted_arguments = []
            for arg_name, arg_value in arguments.items():
                formatted_arguments.append({
                    "argument_name": arg_name,
                    "argument_value": arg_value  # Assuming it's already in the correct format
                })
            final_output.append({
                "tool_name": tool_name,
                "arguments": formatted_arguments
            })

        return final_output
    except json.JSONDecodeError:
        print("Error parsing the JSON output from the model.")
        return None

In [ ]:
# Multi Agentic Framework- using FAISS (Facebook AI Similarity Search), Fuzzywuzzy and Levenshtein distance algorithm

# Embedding function using the new OpenAI API (>=1.0.0)
def embed_text(text):
    response = openai.embeddings.create(  # Correct method for embedding creation
        model="text-embedding-ada-002",
        input=text
    )
    
    # Access the embedding using the attributes of the response object
    embedding = np.array(response.data[0].embedding)  # Updated line
    print(f"Generated embedding for: '{text}'")
    return embedding

# Build FAISS index by embedding the toolset descriptions
def build_faiss_index(toolset):
    print("\nBuilding FAISS index with tool descriptions...")
    print("Toolset structure:", toolset)  # Debugging line to print the structure
    for tool in toolset:
        if isinstance(tool, dict) and 'tool_description' in tool:
            description = tool['tool_description']
            embedding = embed_text(description)
            index.add(np.array([embedding]))
            tool['embedding'] = embedding
        else:
            print(f"Invalid tool format: {tool}")  # Debugging invalid entries
    print("FAISS index built successfully.")


# Use FAISS to find the best-matching tools for a query
def find_best_tool(query, toolset):
    query_embedding = embed_text(query)
    distances, indices = index.search(np.array([query_embedding]), k=5)  # Get top 5 tools
    
    best_tools = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx < len(toolset):  # Ensure the index is valid
            similarity_score = 1 - dist  # Convert distance to similarity
            print(f"Tool '{toolset[idx]['tool_name']}' similarity score: {similarity_score}")
            if similarity_score > SIMILARITY_THRESHOLD:
                best_tools.append((toolset[idx], similarity_score))
        else:
            print(f"Warning: Index {idx} is out of range for toolset.")
    
    # If no tools pass the similarity threshold, apply fuzzy matching
    if len(best_tools) == 0:
        print("No tools found with high similarity, applying fuzzy matching.")
        return find_best_tool_fuzzy(query, toolset)
    
    return best_tools

# Fuzzy matching as a fallback when FAISS doesn't find high-scoring tools
def find_best_tool_fuzzy(query, toolset):
    best_tool = None
    best_score = 0
    for tool in toolset:
        score = fuzz.partial_ratio(query, tool['tool_description'])
        print(f"Fuzzy score for tool '{tool['tool_name']}': {score}")
        if score > FUZZY_THRESHOLD and score > best_score:
            best_tool = tool
            best_score = score
    return [best_tool] if best_tool else []


# Check if tool chaining is required based on tool output/input compatibility
def check_tool_chaining(best_tools):
    # If multiple tools have high similarity, compare their output and input types
    for i in range(len(best_tools) - 1):
        current_tool = best_tools[i][0]
        next_tool = best_tools[i + 1][0]
        
        # Check if the output of the current tool matches the input of the next tool
        output_type = current_tool['output']['arg_type']
        next_tool_inputs = [arg['arg_type'] for arg in next_tool['args']]
        
        if output_type in next_tool_inputs:
            print(f"Tool chaining required: '{current_tool['tool_name']}' -> '{next_tool['tool_name']}'")
            return True
    return False

# Query Parsing Agent: Parses the query and identifies the main components (tools, arguments)
def query_parser_agent(query, toolset):
    best_tools = find_best_tool(query, toolset)
    
    if check_tool_chaining(best_tools):
        return chain_tools(best_tools)
    else:
        return format_output(best_tools)

# Tool Selector Agent: Selects tools based on embeddings and FAISS
def tool_selector_agent(query, toolset):
    return find_best_tool(query, toolset)

# Chaining Coordinator Agent: Coordinates the chaining of tools based on output/input matching
def chain_tools(best_tools):
    chained_tools = []
    for i, tool_info in enumerate(best_tools):
        tool, similarity = tool_info
        tool_data = {
            "tool_name": tool['tool_name'],
            "arguments": []  # Populate based on the query
        }
        # If chaining, use $$PREV[i] for dependencies
        if i > 0:
            tool_data['arguments'].append({
                "argument_name": "created_by",
                "argument_value": f"$$PREV[{i-1}]"
            })
        chained_tools.append(tool_data)
    
    return chained_tools

# Result Aggregator Agent: Formats the tool(s) selected into the required JSON structure
def format_output(best_tools):
    formatted_output = []
    for tool_info in best_tools:
        tool, similarity = tool_info
        # Example: Add argument names and values based on the query logic
        formatted_output.append({
            "tool_name": tool['tool_name'],
            "arguments": [{"argument_name": "example_argument", "argument_value": "example_value"}]
        })
    print("\nFinal tool selection result:")
    print(json.dumps(formatted_output, indent=4))  # Ensure proper JSON formatting
    return formatted_output

In [ ]:
# # Example queries
# query_1 = "Find issues or tasks created by users ’DEVU-123’ or ’DEVU-456’"
# # Build FAISS index with the toolset
# build_faiss_index(toolset)
# # Process the queries
# print("\nProcessing Query 1:")
# result_1 = query_parser_agent(query_1, toolset)

In [ ]:
#Multi Agentic Framework with full GPT3.5 Calls

# Agent to select the best tools based on the user query and tool descriptions
def tool_selection_agent(query):
    # Convert toolset to JSON string for the prompt
    toolset_json = json.dumps(toolset, indent=2)

    prompt = f"""
    You are an intelligent assistant. Based on the user query, identify the most appropriate tool(s) from the provided toolset descriptions.
    Map the corresponding arguments to keywords found in the query.
    
    User Query: "{query}"
    
    Toolset:
    {toolset_json}
    
    Provide the selected tool(s) and their respective arguments in this format:
    - tool_name: "name_of_the_tool"
      arguments: [
          {{"argument_name": "arg1", "argument_value": "value1"}},
          {{"argument_name": "arg2", "argument_value": ["value2a", "value2b"]}}
      ]
    """

    # Use the new API method correctly
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response['choices'][0]['message']['content'].strip()

# Agent to determine how to chain the selected tools
def chaining_coordinator_agent(selected_tools):
    prompt = f"""
    You are an intelligent assistant. Given the selected tools and their descriptions, determine how they should be chained together.
    
    Selected Tools:
    {selected_tools}
    
    Provide the chaining logic in normal text format.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response['choices'][0]['message']['content'].strip()

# Main function to process the user query
def process_query(query):
    # Step 1: Tool Selection
    selected_tools = tool_selection_agent(query)
    print("Selected Tools and Mapped Arguments:")
    print(selected_tools)
    
    # Convert selected tools to a list for further processing
    try:
        selected_tools_list = eval(selected_tools)  # Use eval carefully, ensure the format is trusted
    except Exception as e:
        print("Failed to parse selected tools. Check the output format.")
        print(e)
        return
    
    # Check the length of selected tools
    if len(selected_tools_list) == 1:
        tool = selected_tools_list[0]
        print(f"Selected Tool: {tool['tool_name']}")
        print("Mapped Arguments:")
        for arg in tool['arguments']:
            print(f"  {arg['argument_name']}: {arg['argument_value']}")
    else:
        # More than one tool selected, proceed to chaining coordinator
        final_output = chaining_coordinator_agent(selected_tools)
        print("Chaining Logic:")
        print(final_output)


In [ ]:
# # Example user query
# user_query = "List issues with ’blocker’ severity categorized as tickets."
# process_query(user_query)

In [7]:

# Example query
query_1 = "Fetch ’p3’ priority work items that need customer response for ’REV-333’"
raw_result = find_best_tool(query_1, toolset)
result_json = parse_tool_recommendation(raw_result)

print("Query 1 Result:", json.dumps(result_json, indent=2))

Raw output from the model: {'query': 'Fetch ’p3’ priority work items that need customer response for ’REV-333’', 'toolset': 'who_am_i: Returns the id of the current user\nget_sprint_id: Returns the ID of the current sprint\nworks_list: Returns a list of work items matching the request\nsummarize_objects: Summarizes a list of objects.\nprioritize_objects: Returns a list of objects sorted by priority.\nadd_work_items_to_sprint: Adds the given work items to the sprint\nget_similar_work_items: Returns a list of work items that are similar to the given work item\nsearch_object_by_name: Given a search string, returns the id of a matching object in the system of record.\ncreate_actionable_tasks_from_text: Given a text, extracts actionable insights and creates tasks for them.', 'text': '{\n    "works_list": {\n        "priority": "p3",\n        "requirement": "customer response",\n        "sprint": "REV-333"\n    }\n}'}
Query 1 Result: [
  {
    "tool_name": "works_list",
    "arguments": [
  